## 0. Install Dependencies

## 1. Test Random Environment with OpenAI Gym

In [1]:
import tensorflow
import keras

2024-04-12 16:24:42.824997: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import gym # using older version
import random
import numpy as np
import tensorflow
import keras

In [3]:
print(gym.__version__)
print(tensorflow.__version__)
print(keras.__version__)

0.17.3
2.13.0
2.13.1


In [4]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0] # consists of 4 states: Cart Position, Cart Velocity, Pole Angle, Pole Angular Velocity
actions = env.action_space.n # consists of 2 actions: push cart left, push cart right

In [5]:
episodes = 10
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0

    while not done: # if environemnt is terminated or truncated, exit loop
        env.render() # allow us to see the cart movement 
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:22.0
Episode:2 Score:12.0
Episode:3 Score:29.0
Episode:4 Score:16.0
Episode:5 Score:24.0
Episode:6 Score:18.0
Episode:7 Score:14.0
Episode:8 Score:15.0
Episode:9 Score:16.0
Episode:10 Score:21.0


## 2. Build Agent with Keras-RL

In [6]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [7]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 50000, window_length = 1)
    dqn = DQNAgent(model = model, memory = memory, policy = policy,
                  nb_actions = actions, nb_steps_warmup = 10, target_model_update = 1e-2)
    return dqn

## 3. Create a Deep Learning Model with Keras

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers.legacy import Adam

In [10]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1,states)))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    return model

In [11]:
model = build_model(states, actions)

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 24)                120       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770 (3.01 KB)
Trainable params: 770 (3.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics = ['mae'])
dqn.fit(env, nb_steps = 50000, visualize = False, verbose = 1)

/opt/anaconda3/lib/python3.11/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-04-12 16:25:48.414700: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_1_3/kernel/Assign' id:495 op device:{requested: '', assigned: ''} def:{{{node dense_1_3/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_1_3/kernel, dense_1_3/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 10:00 - reward: 1.0000

/opt/anaconda3/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2024-04-12 16:25:48.642503: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_2/BiasAdd' id:75 op device:{requested: '', assigned: ''} def:{{{node dense_2/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_2/MatMul, dense_2/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-04-12 16:25:48.663021: W tensorflow/c/c_api.cc:304] Operation '{name:'total_1/Assign' id:662 op device:{requested: '', assigned: ''} def:{{{node total_1/Assign}} = AssignVariableOp[_has_manual_control_dependen

10000/10000 [==============================] - 38s 4ms/step - reward: 1.0000
102 episodes - episode_reward: 97.471 [10.000, 200.000] - loss: 2.511 - mae: 19.019 - mean_q: 38.486

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 36s 4ms/step - reward: 1.0000
50 episodes - episode_reward: 197.840 [176.000, 200.000] - loss: 6.369 - mae: 38.966 - mean_q: 78.622

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 34s 3ms/step - reward: 1.0000
51 episodes - episode_reward: 198.902 [176.000, 200.000] - loss: 6.009 - mae: 42.546 - mean_q: 85.540

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 35s 4ms/step - reward: 1.0000
50 episodes - episode_reward: 198.640 [174.000, 200.000] - loss: 5.394 - mae: 41.166 - mean_q: 82.517

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 36s 4ms/step - reward: 1.0000
done, took 179.002 seconds


In [18]:
scores = dqn.test(env, nb_episodes = 100, visualize = False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [19]:
_ = dqn.test(env, nb_episodes = 5, visualize = True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200


## 4. Reloading Agent from Memory

In [20]:
dqn.save_weights('dqn_weights.h5f', overwrite = True)

In [21]:
# delete everything and reload
del model
del dqn
del env

In [22]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics = ['mae'])

2024-04-12 16:39:22.455081: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_5_2/kernel/Assign' id:1274 op device:{requested: '', assigned: ''} def:{{{node dense_5_2/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_5_2/kernel, dense_5_2/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [23]:
dqn.load_weights('dqn_weights.h5f')

2024-04-12 16:40:03.725982: W tensorflow/c/c_api.cc:304] Operation '{name:'total_6/Assign' id:1433 op device:{requested: '', assigned: ''} def:{{{node total_6/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](total_6, total_6/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [24]:
_ = dqn.test(env, nb_episodes = 5, visualize = True)

Testing for 5 episodes ...


2024-04-12 16:40:24.227378: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_5_1/BiasAdd' id:1210 op device:{requested: '', assigned: ''} def:{{{node dense_5_1/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_5_1/MatMul, dense_5_1/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
